# Analyzing The Vasicek Model

In [1]:
import numpy as np
from scipy.stats import norm, lognorm

## The Ornstein Uhlenbeck Model
The general Ornstein-Uhlenbeck model is a mean reverting Stochatic Differential Equation typicially written in the form seen below:

$dX_t = \theta(\mu - X_t)dt + \sigma dW_t$

The model has several applications in the sciences (evolutionary biology and physics) as well as finance through the **Vasicek Model** described below.

## The Vasicek model.
The vasicek model is a common interest rate model for pricing interest rate securities.  The model is a very simple Ornstein Uhlenbeck process where $r_t$ is the short rate and $\bar{r}$ is the average of the historical short rate.

$dr_t = \gamma(\bar{r} - r_t)dt + \sigma dW_t$

### Properties of Vasicek Model:

$r_t = r_0e^{-\gamma t} + \bar{r}(1 - e^{-\gamma t}) + \sigma \int_{0}^{t} e^{-\gamma (t-s)} dWs$

$E[r_t|r_0] = r_0e^{-\gamma t} + \bar{r}(1 - e^{-\gamma t})$

$Var(r_t) = \frac{\sigma^2}{2\gamma}(1 - e^{-2\gamma t})$

* Short rate is normally distributed with above mean and variance between time 0 and t

### Pros of the Vasicek Model
* Mean Reverting
* Closed form pricing and expectation
* Relatively simple model to understand

### Cons of Vasicek Model:
* All rates in term structure are perfectly correlated (true for all 1 factor models)
* Constant volatility
* Allows Arbitrage
* Possibility of negative interest rates

### Extensions of Vasicek
* The Ho Lee and Hull White models are essentially extensions of the Vasicek model, but have the added advantage of being able to replicate term structures exactly with additional free parameters ($\theta_t$ and  $\sigma_t$).
* Hull white is the most flexible, but can be improved by using a two factor model that accounts for level and curvature of term structure
* To further improve Hull White, you can add a 3rd factor for the slope of the term structure

In [2]:
def vasicek_expectation(gamma, t, r_bar, r_0):
    return r_0*np.exp(-gamma*t) + r_bar*(1 - np.exp(-gamma*t))

def vasicek_variance (sigma, gamma, t):
    return sigma**2/(2*gamma)*(1-np.exp(-2*gamma*t))

## Sample Vasicek Model Calculations
Suppose short rates follow the vasicek model with the average rate is long run average rate is 4% and the reversion speed $\gamma = 0.25$.  If the current rate is 2%, what is the expected short rate in 5 years?

In [3]:
gamma = 0.25
r_bar = 0.04
r_0 = 0.02
t = 5

r_t_expectation = vasicek_expectation(gamma, t, r_bar, r_0)

print(f"Expected rate in {t} years: {r_t_expectation}")

Expected rate in 5 years: 0.034269904062796196


Suppose the volatility of the short rate is 5%, what is the variance of the short rate?

In [4]:
sigma = 0.05

r_t_variance = vasicek_variance(sigma, gamma, t)
print(f"Variance of the rate in {t} years: {r_t_variance}")

Variance of the rate in 5 years: 0.004589575006880507


What is the probability the rate will be negative in 4 years?

In [5]:
norm.cdf(-r_t_expectation/r_t_variance ** 0.5)

0.3064789746939356

Since $r_5$ is normally distributed with a mean of 0.03427 and variance of 0.004590, the probability of negative interest rates is:

$\Phi(\frac{0 - 0.03427}{\sqrt{0.004590}}) = 0.30648$

### Vasciek SDE Solution
To solve for $r_t$ in the vasicek model:

$dr_t = \gamma(\bar{r} - r_t)dt + \sigma dW_t$

Let 
$F_t = r_te^{\gamma t}$

By Itos lemma

$dF = (\frac{\partial F}{\partial t} + a_t\frac{\partial F}{\partial r} + \frac{1}{2}s_t^{2}\frac{\partial^2 F}{\partial r_t})dt + s_tdW_t$

$\frac{\partial F}{\partial r} = e^{\gamma t}$

$\frac{\partial^2 F}{\partial r^2} = 0$

$\frac{\partial F}{\partial t} = \gamma r_te^{\gamma t}$

Under Vasicek:

$a_t = \gamma(\bar{r} - r_t)$ and $s_t = \sigma$

Plugging into Itos Lemma:

$d(r_te^{\gamma t}) = (\gamma r_te^{\gamma t} + \gamma(\bar{r} - r_t)e^{\gamma t})dt + e^{\gamma t}\sigma dW_t$

$d(r_te^{\gamma t}) = \gamma e^{\gamma t}(r_t + (\bar{r} - r_t))dt + e^{\gamma t}\sigma dW_t$

$d(r_te^{\gamma t}) = \gamma \bar{r}e^{\gamma t}dt + e^{\gamma t}\sigma dW_t$

Integrating both sides from 0 to t, we get:

$r_te^{\gamma t} - r_0 = \bar{r}(e^{\gamma t} - 1) + \sigma \int_{0}^{t} e^{\gamma s} dWs$

Multiplying both sides by $e^{-\gamma t}$ and isolating $r_t$ gives us the SDE for the Vasicek model:

$r_t = r_0e^{-\gamma t} + \bar{r}(1 - e^{-\gamma t}) + \sigma \int_{0}^{t} e^{-\gamma (t-s)} dWs$

In [6]:
print(vasicek_expectation(0.15, 5, 0.05, 0.03))

0.040552668945179705
